# VITS

In [1]:
# Download model, and save
# https://huggingface.co/kakao-enterprise/vits-ljs
from transformers import VitsModel, AutoTokenizer
import torch

models_folder_path = "./../models"
model_name = "kakao-enterprise/vits-ljs"
model = VitsModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.save_pretrained(models_folder_path+ "/local_vits")
tokenizer.save_pretrained(models_folder_path+ "/local_vits")

2025-04-04 04:40:59.545201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743734459.641191  174133 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743734459.668091  174133 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743734459.884240  174133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743734459.884277  174133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743734459.884280  174133 computation_placer.cc:177] computation placer alr

('./../models/local_vits/tokenizer_config.json',
 './../models/local_vits/special_tokens_map.json',
 './../models/local_vits/vocab.json',
 './../models/local_vits/added_tokens.json')

In [2]:
from transformers import VitsModel, AutoTokenizer
import time

start = time.time()
text = "specifies."
text += "specifies the required processes implemented in the engineering activities."
# text += "specifies the required processes implemented in the engineering activities that result in requirements for systems and software products (including services) throughout the life cycle"
# text += "provides guidelines for applying the requirements and requirements-related processes described in ISO/IEC/IEEE 15288 and ISO/IEC/IEEE 12207;"
text = text.replace("(", "")
text = text.replace(")", "")

print(time.time() - start)
inputs = tokenizer(text, return_tensors="pt")

print(time.time() - start)
with torch.no_grad():
    output = model(**inputs).waveform

print(time.time() - start)

Audio(output, rate=model.config.sampling_rate)
model = VitsModel.from_pretrained("./local_vits")
tokenizer = AutoTokenizer.from_pretrained("./local_vits")

0.00010204315185546875


NameError: name 'tokenizer' is not defined

# SpeechT5

## Download

## Download Model SpeechT5

In [15]:
from transformers import VitsModel, AutoTokenizer
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech


models_folder_path = "./../models/speecht5"
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

processor.save_pretrained(models_folder_path+ "/processor")
model.save_pretrained(models_folder_path+ "/model")

/home/dawid/mydir/fastapi1/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 1876}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


## Download dataset

In [12]:
from datasets import load_dataset, load_from_disk

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

datasets_folder_path = "./../datasets/embeddings_dataset"
embeddings_dataset.save_to_disk(datasets_folder_path)

Saving the dataset (0/1 shards):   0%|          | 0/7931 [00:00<?, ? examples/s]

## Download vocoder

In [13]:
from transformers import SpeechT5HifiGan

models_folder_path = "./../models/hifigan"
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

vocoder.save_pretrained(models_folder_path+ "/vocoder")

## Run

In [14]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

In [3]:
inputs = processor(text="Don't count the days, make the days count.", return_tensors="pt")

In [11]:
from datasets import load_dataset

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

import torch
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [5]:
spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)

In [6]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

In [7]:
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [8]:
from IPython.display import Audio

Audio(speech, rate=16000)